In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools.analysis as analysisTools
import timeit
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
# Main parameters
numparticles = 2
boxsize = 6 #10 #8 #6 #5 
D = 1.0
Drot = 1.0
lagtime = 150 #150 #75 #75 #75
rootDirectory = '../../data/patchyProtein/first_passage_times/'
#rootDirectory = '../../data/testPatchyProtein/first_passage_times/'

In [ ]:
# Color definitions
c1 = 'xkcd:bright orange' # 'darkorange' #'C1' # '#ff3300' # for benchmark
c2 = 'royalblue' #'royalblue' # 'C0' # '#0099ff' # for MSM/RD
# Alpha transparency definition
a1 = 0.2 #0.2
a2 = 0.9 #0.7

## Load unbound to bound FPT data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 600
filename1 = rootDirectory + 'patchyProteinFPTs_2bound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
#filename1 = rootDirectory + 'testNew7_ALL_PatchyProteinFPTs_2bound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fpts = []
with open(filename1) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fpts.append(time)
            
# Normalize weights
weights = np.ones_like(fpts)/float(len(fpts))
max(fpts)

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 600
filename = rootDirectory + 'MSMRDpatchyProteinFPTs_2bound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
#filename = rootDirectory + 'testNew7_ALL_MSMRDpatchyProteinFPTs_2bound_trajs' + \
#str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfpts = []
with open(filename) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfpts.append(time)
            
# Normalize weights
#MSMRDfpts = [x for x in MSMRDfpts if x <= max(fpts)]
weightsMSMRD = np.ones_like(MSMRDfpts)/float(len(MSMRDfpts))
max(MSMRDfpts)

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
calculateRates = True
numBootsrapSamples = 2000
mfpt, fptstd = analysisTools.bootstrapping(fpts, numBootsrapSamples)
print("Raw MFPT: ", np.array(fpts).mean())
print("Bootstrapping mean and std: ", mfpt, fptstd)
# Compuate mean and standard deviation of bootstrapped samples of MSMRD simulation
MSMRDmfpt, MSMRDfptstd = analysisTools.bootstrapping(MSMRDfpts, numBootsrapSamples)
print("Raw MSMRD MFPT: ",np.array(MSMRDfpts).mean())
print("MSMRD bootstrapping mean and std: ", MSMRDmfpt, MSMRDfptstd)
if calculateRates:
    mkon = 1.0/mfpt
    MSMRDmkon = 1.0/MSMRDmfpt
    konstd = fptstd/(mfpt*mfpt)
    MSMRDkonstd = MSMRDfptstd/(MSMRDmfpt*MSMRDmfpt)
    print("On rates:")
    print("Bootstrapping mean and std:", mkon, konstd)
    print("Bootstrapping MSMRD mean and std (A):", MSMRDmkon, MSMRDkonstd)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, ax1 = plt.subplots(nrows=1, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])
hbins = range(0, 500,15)
#hbins = range(0, 2000,60)
#hbins = np.arange(0, 10,0.5)


# Plot A to unbound histogram
ax1.hist(fpts, bins = hbins, alpha=a1, color=c1, weights = weights );
ax1.hist(fpts, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step', 
         facecolor = 'None', weights = weights, label =r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfpts, bins = hbins, alpha=a1, color=c2, weights = weightsMSMRD);
ax1.hist(MSMRDfpts, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step', 
         weights = weightsMSMRD, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=r'$\mathrm{Unbound} \rightarrow \mathrm{Bound}$', title_fontsize = fsize, fancybox=True)
ax1.text(100, 0.15, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfpt, fptstd), fontsize=fsize, color=c1)
ax1.text(120, 0.13, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfpt, MSMRDfptstd), fontsize=fsize, color=c2)
ax1.set_yticks(np.arange(0, 0.18, step=0.1));
ax1.set_xlim([0,500])
#ax1.set_xlim([0,2000])
#ax1.set_xlim([0,10])
#ax1.set_ylim([0,0.18])

plt.savefig('toBound_patchyProtein_FPTs.pdf', bbox_inches='tight')

## Load bound to unbound FPT data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 5000
filename = rootDirectory + 'patchyProteinFPTs_2unbound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fpts2unbound = []
with open(filename) as file:
    
    for line in file:
        state, dummy1, dummy2, dummy3, dummy4, dummy5, time = line.split(" ")
        time = float(time)
        fpts2unbound.append(time)
        
# Normalizing weights
weights2 = np.ones_like(fpts2unbound)/float(len(fpts2unbound))

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 5000
filename = rootDirectory + 'MSMRDpatchyProteinFPTs_2unbound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfpts2unbound = []
with open(filename) as file:
    for line in file:
        state, dummy1, dummy2, dummy3, dummy4, dummy5, time = line.split(" ")
        time = float(time)
        MSMRDfpts2unbound.append(time)
        
# Normalizing weights
weightsMSMRD2 = np.ones_like(MSMRDfpts2unbound)/float(len(MSMRDfpts2unbound))

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
numBootsrapSamples = 2000
mfpt2unbound, fpt2unboundstd = analysisTools.bootstrapping(fpts2unbound, numBootsrapSamples)
print("Raw MFPT: ", np.array(fpts2unbound).mean())
print("Bootstrapping mean and std: ", mfpt2unbound, fpt2unboundstd)
MSMRDmfpt2unbound, MSMRDfpt2unboundstd = analysisTools.bootstrapping(MSMRDfpts2unbound, numBootsrapSamples)
print("Raw MSMRD MFPT: ",np.array(MSMRDfpts2unbound).mean())
print("MSMRD bootstrapping mean and std: ", MSMRDmfpt2unbound, MSMRDfpt2unboundstd)
# Calculate rates
mkoff = 1.0/mfpt2unbound
mkoffMSMRD = 1.0/MSMRDmfpt2unbound
koffstd = fpt2unboundstd/(mfpt2unbound*mfpt2unbound)
koffstdMSMRD = MSMRDfpt2unboundstd/(MSMRDmfpt2unbound*MSMRDmfpt2unbound)
print("Off rates:")
print("Bootstrapping mean and std:", mkoff, koffstd)
print("Bootstrapping MSMRD mean and std:", mkoffMSMRD, koffstdMSMRD)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, ax1 = plt.subplots(nrows=1, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
#hbins = np.arange(0, 15, 0.5)
hbins = np.arange(0, 10, 0.33333333)
#hbins = np.arange(0, 20, 0.66666666)

# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])

# Plot A to unbound histogram
ax1.hist(fpts2unbound, bins = hbins, alpha=a1, color=c1, weights = weights2);
ax1.hist(fpts2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weights2, label = r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfpts2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsMSMRD2);
ax1.hist(MSMRDfpts2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsMSMRD2, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=r'$ \mathrm{Bound} \rightarrow \mathrm{Unbound}$', title_fontsize = fsize, fancybox=True)
ax1.text(3.2, 0.105, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfpt2unbound, fpt2unboundstd), fontsize=fsize, color=c1)
ax1.text(3.4, 0.09, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfpt2unbound, MSMRDfpt2unboundstd), 
         fontsize=fsize, color=c2)
#ax1.set_xlim([0,15])
#ax1.set_ylim([0,0.2])
ax1.set_xlim([0,10])
ax1.set_ylim([0,0.18])
ax1.set_yticks(np.arange(0, 0.2, step=0.1));


plt.savefig('toUnbound_patchyProtein_FPTs.pdf', bbox_inches='tight')

## UP UNTIL HERE MODIFICATIONS FOR PATCHY PROTEINS

## Load bound to bound FPT data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 1000
boundStates = [1,2,3,4,5,6]
fptsB2B = np.zeros([len(boundStates),len(boundStates),numTrajs])
weightsB2B = np.zeros([len(boundStates),len(boundStates),numTrajs])
for i in boundStates:
    for j in boundStates:
        if i != j:
            filenameB2B = rootDirectory + 'patchyProteinFPTs_' + str(i) + 'to' + str(j) \
            + '_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
            with open(filenameB2B) as file:
                k=0
                for line in file:
                    initState, endState, time = line.split(" ")
                    time = float(time)
                    fptsB2B[i-1,j-1,k] = time
                    k += 1

            # Normalizing weights
            weightsB2B[i-1,j-1] = np.ones_like(fptsB2B[i-1,j-1])/float(len(fptsB2B[i-1,j-1]))

In [ ]:
# Load FPT data obtained with MSMRD integrator
# Load benchmark FPT data computed by script
MSMRDfptsB2B = np.zeros([len(boundStates),len(boundStates),numTrajs])
MSMRDweightsB2B = np.zeros([len(boundStates),len(boundStates),numTrajs])
for i in boundStates:
    for j in boundStates:
        if i != j:
            filenameB2B = rootDirectory + 'MSMRDpatchyProteinFPTs_' + str(i) + 'to' + str(j) \
            + '_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
            with open(filenameB2B) as file:
                k=0
                for line in file:
                    initState, endState, time = line.split(" ")
                    time = float(time)
                    MSMRDfptsB2B[i-1,j-1,k] = time
                    k += 1

            # Normalizing weights
            MSMRDweightsB2B[i-1,j-1] = np.ones_like(MSMRDfptsB2B[i-1,j-1])/float(len(MSMRDfptsB2B[i-1,j-1]))

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
numBootsrapSamples = 2000
mfptsB2B = np.zeros([len(boundStates),len(boundStates)])
MSMRDmfptsB2B = np.zeros([len(boundStates),len(boundStates)])
fptB2Bstd = np.zeros([len(boundStates),len(boundStates)])
MSMRDfptB2Bstd = np.zeros([len(boundStates),len(boundStates)])
mfptsB2BDiff = np.zeros([len(boundStates),len(boundStates)])
print("Begins boostrapping:") 
print("Transition     MFPT       stddev     MSMRD-MFPT  MSMRD-stddev    Difference")
for i in boundStates:
    for j in boundStates:
        if i != j:
            mfptsB2B[i-1,j-1], fptB2Bstd[i-1,j-1] = analysisTools.bootstrapping(fptsB2B[i-1,j-1], numBootsrapSamples)
            #MSMRDmfptsB2B[i-1,j-1], MSMRDfptB2Bstd[i-1,j-1] = analysisTools.bootstrapping(MSMRDfptsB2B[i-1,j-1], numBootsrapSamples)
            mfptsB2BDiff[i-1,j-1] = np.abs(mfptsB2B[i-1,j-1] - MSMRDmfptsB2B[i-1,j-1])
            print(str(i) + " to " + str(j) + ":    {:10.4f} {:10.2f} {:10.4f} {:10.2f} {:18.4f}"
                  .format(mfptsB2B[i-1,j-1], fptB2Bstd[i-1,j-1], MSMRDmfptsB2B[i-1,j-1], 
                          MSMRDfptB2Bstd[i-1,j-1], mfptsB2BDiff[i-1,j-1]))

In [ ]:
# Grid plot for histograms
fig, ax = plt.subplots(nrows=6, ncols=5, sharex=True, sharey=True, figsize=(14,10))
fig.subplots_adjust(hspace=0, wspace=0)
fsize = 20
fsize2 = 12
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
#hbins = np.arange(0, 500, 15)
hbins = np.arange(0, 600, 18)
# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])
# Loop over plotting grid
for i in range(6):
    for j in range(5):
        if j < i:
            jj = j
        else:
            jj = j+1
        ax[i,j].hist(fptsB2B[i,jj], bins = hbins, alpha=a1, color=c1, weights = weightsB2B[i,jj]);
        ax[i,j].hist(fptsB2B[i,jj], bins = hbins, alpha=a2, edgecolor=c1, lw=1, histtype='step',
         facecolor='None', weights = weightsB2B[i,jj], label = r'$\mathrm{Benchmark}$');
        ax[i,j].hist(MSMRDfptsB2B[i,jj], bins = hbins, alpha=a1, color=c2, weights = MSMRDweightsB2B[i,jj]);
        ax[i,j].hist(MSMRDfptsB2B[i,jj], bins = hbins, alpha=a2, edgecolor=c2, lw=1, histtype='step',
         facecolor='None', weights = MSMRDweightsB2B[i,jj], label =r'$\mathrm{MSM/RD}$');
        ax[i,j].text(350,0.1, r'{:d} to {:d}'.format(i+1, jj+1), fontsize=fsize, family="serif")
        ax[i,j].text(200,0.08, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptsB2B[i,jj] , fptB2Bstd[i,jj]), 
                     fontsize=fsize2, color=c1)
        ax[i,j].text(220,0.06, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptsB2B[i,jj], MSMRDfptB2Bstd[i,jj]), 
                     fontsize=fsize2, color=c2)
        # Add legend
        if i == 0 and j == 4:
            ax[i,j].legend(fontsize=fsize, fancybox=True, framealpha=1.0, 
                           bbox_to_anchor=(1.4, 1.55))
        # Set plotlimits
        ax[i,j].set_xlim([0,600])
        #ax[i,j].set_ylim([0,0.23])
        ax[i,j].set_ylim([0,0.15])
        #ax[i,j].set_xticks(np.arange(0, 500, step=300));
        #ax[i,j].set_yticks(np.arange(-0.1, 0.3, step=0.2));
        
#plt.savefig('bound2bound__patchyProtein_FPTs.pdf', bbox_inches='tight')

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
hbins = np.arange(0, 500, 15)

# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])

# Plot A to B histogram
ax1.hist(fptsA2B, bins = hbins, alpha=a1, color=c1, weights = weightsA2B);
ax1.hist(fptsA2B, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsA2B, label = r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfptsA2B, bins = hbins, alpha=a1, color=c2, weights = weightsA2BMSMRD);
ax1.hist(MSMRDfptsA2B, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsA2BMSMRD, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=r'$A \rightarrow B$', title_fontsize = fsize, fancybox=True)
ax1.text(60, 0.18, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptA2B, fptA2Bstd), fontsize=fsize, color=c1)
ax1.text(70, 0.13, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptA2B, MSMRDfptA2Bstd), fontsize=fsize, color=c2)
ax1.set_xlim([0,500])
ax1.set_ylim([0,0.3])
ax1.set_yticks(np.arange(0, 0.2, step=0.1));

# Plot B to A histogram
ax2.hist(fptsB2A, bins = hbins, alpha=a1, color=c1, weights = weightsB2A);
ax2.hist(fptsB2A, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB2A, label = r'$\mathrm{Benchmark}$');
ax2.hist(MSMRDfptsB2A, bins = hbins, alpha=a1, color=c2, weights = weightsB2AMSMRD);
ax2.hist(MSMRDfptsB2A, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsB2AMSMRD, label =r'$\mathrm{MSM/RD}$');
ax2.legend(fontsize=fsize, title=r'$B \rightarrow A$', title_fontsize = fsize, fancybox=True)
ax2.text(60, 0.18, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptB2A, fptB2Astd), fontsize=fsize, color=c1)
ax2.text(70, 0.13, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptB2A, MSMRDfptB2Astd), fontsize=fsize, color=c2)
ax2.set_xlim([0,500])
ax2.set_ylim([0,0.3])
ax2.set_yticks(np.arange(0, 0.2, step=0.1));

#plt.savefig('bound2bound_FPTs.pdf', bbox_inches='tight')

## Load all unbound to bound FPT data for $K_{on}$ vs concentration plot

In [ ]:
# Load all data
numTrajs = 5000
boxsizes = [6, 7, 8, 9, 10, 11, 12]
numBoxes = len(boxsizes)
fpt = [[] for i in range(numBoxes)]
MSMRDfpt = [[] for i in range(numBoxes)]
for i, boxsize in enumerate(boxsizes):
    filename1 = rootDirectory + '/patchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
    filename2 = rootDirectory + 'MSMRDpatchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
    with open(filename1) as file:
        for line in file:
            state, time = line.split(" ")
            fpt[i].append(float(time))
    with open(filename2) as file:
        for line in file:
            state, time = line.split(" ")
            MSMRDfpt[i].append(float(time))

### Calculation of FPTs and comparisons plots

In [ ]:
# Calculate meand and bootstrapping error
numBootsrapSamples = 1000
numValues = 250
meanKon = np.zeros(numBoxes)
meanKonMSMRD = np.zeros(numBoxes)
stdDevKon = np.zeros(numBoxes)
stdDevKonMSMRD = np.zeros(numBoxes)
errorKon = np.zeros(numBoxes)
errorKonMSMRD = np.zeros(numBoxes)
for i, boxsize in enumerate(boxsizes):
    # Calculate bootstrapping samples of mean first passage times
    mfpt, fptStdDev = analysisTools.bootstrapping(fpt[i], numBootsrapSamples, numValues)
    MSMRDmfpt, MSMRDfptStdDev = analysisTools.bootstrapping(MSMRDfpt[i], numBootsrapSamples, numValues)
    # Calculate on rate corresponding to bootstrapping samples
    meanKon[i], stdDevKon[i] = 1.0/mfpt, fptStdDev/(mfpt*mfpt)
    meanKonMSMRD[i], stdDevKonMSMRD[i] = 1.0/MSMRDmfpt, MSMRDfptStdDev/(MSMRDmfpt*MSMRDmfpt)
    # Percentual errors
    errorKon[i] = stdDevKon[i] #1.96*stdDevKon[i]/numBootsrapSamples
    errorKonMSMRD[i] = stdDevKonMSMRD[i] #1.96*stdDevKonMSMRD[i]/numBootsrapSamples

In [ ]:
# Plot on rates vs concentration:
fig = plt.figure()
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 2
concentrations = 1.0/pow(np.array(boxsizes),3)

# Plots with shaded regions as error bars
ax.plot(concentrations, meanKon, marker='o', markersize=6, linewidth=linewidth , color=c1, 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrations, meanKon-errorKon, meanKon+errorKon, alpha=0.3, edgecolor=c1, facecolor=c1)

# Traditional error bar plots
ax.errorbar(concentrations, meanKonMSMRD, yerr=errorKonMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color=c2, label=r'$\mathrm{MSM/RD}$')

ax.set_yscale('log')
ax.set_xlabel(r'$\mathrm{Concentration}$ ($1/\mathrm{Volume}$)', fontsize = fsize+4)
ax.set_ylabel(r'$\mathrm{log}(k_\mathrm{on}$)', fontsize = fsize+6)
ax.set_yticks([0.001,.01])
#ax.set_ylim([0,0.12])
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)
plt.savefig('kon_dimer.pdf', bbox_inches='tight')

In [ ]:
# Plot equilibrium constant vs concentration
fig = plt.figure()
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 2
concentrations = 1.0/pow(np.array(boxsizes),3)

# Calculate Keq and errors
Keq = meanKon/meanKoff
KeqMSMRD = meanKonMSMRD/meanKoffMSMRD
errorKeq = Keq*(errorKon/meanKon + koffstd/meanKoff)
errorKeqMSMRD = KeqMSMRD*(errorKonMSMRD/meanKonMSMRD + koffstdMSMRD/meanKoffMSMRD)

# Plots with shaded regions as error bars
ax.plot(concentrations, Keq, marker='o', markersize=6, linewidth=linewidth , color=c1, 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrations, Keq-errorKeq, Keq+errorKeq, alpha=0.3, edgecolor=c1, facecolor=c1)

# Traditional error bar plots
ax.errorbar(concentrations, KeqMSMRD, yerr=errorKeqMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color=c2, label=r'$\mathrm{MSM/RD}$')

ax.set_yscale('log')
ax.set_xlabel(r'$\mathrm{Concentration}$ ($1/\mathrm{Volume}$)', fontsize = fsize+4)
ax.set_ylabel(r'$\mathrm{log}(k_\mathrm{eq}$)', fontsize = fsize+6)
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)
plt.savefig('keq_dimer.pdf', bbox_inches='tight')

In [ ]:
# Plot on rates vs concentration, zoomed
fig = plt.figure()
fsize = 16
ax = fig.add_subplot(111)
linewidth = 2
concentrations = 1.0/pow(np.array(boxsizes),3)

# Plots with shaded regions as error bars
ax.plot(concentrations, meanKon, marker='o', markersize=6, linewidth=linewidth , color='C1', 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrations, meanKon-errorKon, meanKon+errorKon, alpha=0.3, edgecolor='C1', facecolor='C1')
#ax.plot(concentrations, meanKonMSMRD, marker='o', markersize=6, linewidth=0*linewidth, color='C0', 
#        label=r'$\mathrm{MSM/RD}$)
#ax.fill_between(concentrations, meanKonMSMRD-errorKonMSMRD, meanKonMSMRD+errorKonMSMRD, alpha=0.35, 
#                edgecolor='C0', facecolor='C0')

# Traditional error bar plots
#ax.errorbar(concentrations, meanKon, yerr=errorKon, marker='o', markersize=6, elinewidth=2, 
#            linewidth=linewidth , capsize=5, color='C1', label=r'$\mathrm{Benchmark}$')
ax.errorbar(concentrations, meanKonMSMRD, yerr=errorKonMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color='C0', label=r'$\mathrm{MSM/RD}$')
ax.set_yscale('log')
ax.set_xlim([0.0018,0.0047])
ax.set_ylim([0.003,0.018])
ax.set_xticks([], [])
#plt.savefig('kon_dimer_zoom.pdf', bbox_inches='tight')